# RAG (Retrieval-Augmented Generation)

## Load Environment Variables
All the environment variables, APIS keys etc are loaded in the `.env` file. Load it into memory.

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

print("ℹ️  Loaded environment variables from .env file")

ℹ️  Loaded environment variables from .env file


## Preparing Documents

In [2]:
from langchain_text_splitters import CharacterTextSplitter

print("ℹ️  Preparing Documents...")
# Function to load markdown file content
def load_markdown_file(filename):
    """Reads a markdown file and returns its content as a string."""
    try:
        with open(filename, 'r', encoding='utf-8') as file:
            content = file.read()
        return content
    except FileNotFoundError:
        print(f"Error: The file '{filename}' was not found.")
        return None
    except IOError as e:
        print(f"Error reading file '{filename}': {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred while reading '{filename}': {e}")
        return None
    
# Split the text into chunks
text_splitter = CharacterTextSplitter(
    chunk_size=512, 
    chunk_overlap=50
)

chunks = []
print("  => Loading Artificial-Intelligence.md...")
chunks.extend(text_splitter.split_text(load_markdown_file('./documents/Artificial-Intelligence.md')))
print("  => Loading Deep-Learning.md...")
chunks.extend(text_splitter.split_text(load_markdown_file('./documents/Deep-Learning.md')))
print("  => Loading Machine-Learning.md...")
chunks.extend(text_splitter.split_text(load_markdown_file('./documents/Machine-Learning.md')))
print("  => Loading Python.md...")
chunks.extend(text_splitter.split_text(load_markdown_file('./documents/Python.md')))
print("  => Loading Cloud-Computing.md...")
chunks.extend(text_splitter.split_text(load_markdown_file('./documents/Cloud-Computing.md')))
print("  => Loading Web-Development.md...")
chunks.extend(text_splitter.split_text(load_markdown_file('./documents/Web-Development.md')))

print(f"✅  Loaded and split documents into {len(chunks)} chunks.")
print(f"✅  Sample chunk content:\n{chunks[0][:128]}...")

/home/ebby/05-src/personal/gh-ebbypeter/ELVTR-APAC-AISA1-SelfStudy/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Created a chunk of size 542, which is longer than the specified 512
Created a chunk of size 622, which is longer than the specified 512
Created a chunk of size 662, which is longer than the specified 512
Created a chunk of size 632, which is longer than the specified 512
Created a chunk of size 604, which is longer than the specified 512
Created a chunk of size 542, which is longer than the specified 512
Created a chunk of size 521, which is longer than the specified 512
Created a chunk of size 519, which is longer than the specified 512


ℹ️  Preparing Documents...
  => Loading Artificial-Intelligence.md...
  => Loading Deep-Learning.md...
  => Loading Machine-Learning.md...
  => Loading Python.md...
  => Loading Cloud-Computing.md...
  => Loading Web-Development.md...
✅  Loaded and split documents into 22 chunks.
✅  Sample chunk content:
## Artificial Intelligence...


## Create Embeddings & Store in Vector Database

In [3]:

from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

print("ℹ️  Creating Embeddings and Vector Stores...")
embeddings = OpenAIEmbeddings(
    api_key=os.getenv('OPENAI_API_KEY')
)

vector_store = Chroma.from_texts(
    texts=chunks,
    embedding=embeddings,
    collection_name="knowledge_base"
)
print("✅  Created vector store with embeddings.")

ℹ️  Creating Embeddings and Vector Stores...
✅  Created vector store with embeddings.


## Create a retriever

In [4]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)
print("✅  Configured retriever for similarity search.")

print("ℹ️  Testing retriever with a sample query...")
query = "What is deep learning?"
retrieved_docs = retriever.invoke(query)

print(f"Query: '{query}'")
print(f"✅  Retrieved {len(retrieved_docs)} relevant chunks.")
print("Retrieved Documents:")
for i, doc in enumerate(retrieved_docs):
    print(f"  {i+1}. {doc.page_content}...")

✅  Configured retriever for similarity search.
ℹ️  Testing retriever with a sample query...
Query: 'What is deep learning?'
✅  Retrieved 3 relevant chunks.
Retrieved Documents:
  1. ## Deep Learning...
  2. Deep learning is a subfield of machine learning that uses multilayer artificial neural networks to learn hierarchical representations of data. The “deep” in deep learning refers to networks with many layers (often dozens or more), which successively transform raw inputs into increasingly abstract features. These architectures excel at modeling complex, non‑linear relationships in high‑dimensional data such as images, audio, and text. [en.wikipedia](https://en.wikipedia.org/wiki/Deep_learning)...
  3. Deep learning has driven breakthroughs in image classification, speech recognition, natural language processing, recommendation, and generative media. The trade‑offs are high data requirements, significant computational cost (often GPUs or specialized accelerators), and reduced interpre

## Build a RAG Chain

In [7]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

rag_prompt = ChatPromptTemplate.from_template(
    """
    Use the following retrieved documents to answer the question. If the documents do not contain the answer, say "I don't have that information". 
    Be concise and accurate.
    
    Documents:
    {retrieved_docs}
    
    Question:
    {question}
    
    Answer:
    
    """
)

agent_openai_gpt4o = ChatOpenAI(
    name="OpenAI GPT-4o",
    model="gpt-4o",
    temperature=0.5,
    api_key=os.getenv('OPENAI_API_KEY'),
    organization=os.getenv('OPENAI_ORG_ID') if os.getenv('OPENAI_ORG_ID') else None
) 
print("ℹ️  OpenAI client initialized successfully with GPT-4o mode")

# Create the RAG chain:
def rag_chain(question):
    """Given a question, retrieve relevant documents and generate an answer using the LLM."""
    # Step 1: Retrieve relevant documents
    retrieved_docs = retriever.invoke(question)
    
    # Step 2: Format the retrieved documents into the prompt
    formatted_docs = "\n\n".join([doc.page_content for doc in retrieved_docs])
    #prompt = rag_prompt.format(retrieved_docs=formatted_docs, question=question)
    
    # Step 3: Pass the prompt to the LLM for answer generation
    response = rag_prompt | agent_openai_gpt4o
    answer = response.invoke({
        "retrieved_docs":formatted_docs, 
        "question":question
    })
    
    return answer.content.strip(), formatted_docs

# Test the RAG system
print("\nTesting RAG System:")
print("=" * 60)

test_questions = [
    "Explain machine learning in simple terms",
    "What is deep learning used for?",
    "Tell me about Python",
    "What is Quantum Mechanics?"
]

for q in test_questions:
    print(f"\nQuestion: {q}")
    answer, sources = rag_chain(q)
    print(f"\nAnswer: {answer}")
    print(f"Sources used: {len(sources)} document(s)")
    print("-" * 60)

ℹ️  OpenAI client initialized successfully with GPT-4o mode

Testing RAG System:

Question: Explain machine learning in simple terms

Answer: Machine learning is a type of artificial intelligence where computers learn from data to make predictions or decisions without being specifically programmed for each task. It works by identifying patterns in data and creating a model that can generalize to new, unseen situations. This allows applications like spam email detection, demand forecasting, and credit scoring to improve over time as they process more information.
Sources used: 1580 document(s)
------------------------------------------------------------

Question: What is deep learning used for?

Answer: Deep learning is used for breakthroughs in image classification, speech recognition, natural language processing, recommendation, and generative media.
Sources used: 1043 document(s)
------------------------------------------------------------

Question: Tell me about Python

Answer: Py